In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

if os.path.exists('./file/2.part'):
    shutil.rmtree('./file/2.part')

os.makedirs('./file/2.part')

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests
import json

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import numpy as np

# = = = = = = = = = = = = = = =

import re

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/1.part_number'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

for file in list_file:
    
    # = = = = = = = = = = = = = = =

    output_correct, output_error = pd.DataFrame(), pd.DataFrame()

    # = = = = = = = = = = = = = = =

    input_ = pd.read_excel(f'./file/1.part_number/{file}',
                           header=0,
                           dtype=str).fillna('')
    
    input_['No'] = input_['No'].astype(int)

    print(f'总数量：{len(input_)}')
    print()

    # = = = = = = = = = = = = = = =


    work = Queue()
    for a in range(len(input_)):
        work.put_nowait(a)

    # = = = = = = = = = = = = = = =

    def crawler():
        global output_correct, output_error

        while not work.empty():
            a = work.get_nowait()

            # = = = = = = = = = = = = = = =

            crawler_status = 'error'

            # = = = = = = = = = = = = = = =

            try:
                data = {'dopartsearch': '1',
                        'partsearch[partnum][partsearch_007]': input_.loc[a, 'Part Number'],
                        'partsearch[manufacturer][partsearch_007]': input_.loc[a, 'Manufacturer'],
                        'partsearch[partgroup][partsearch_007]': input_.loc[a, 'Group'],
                        'partsearch[parttype][partsearch_007]': input_.loc[a, 'Type Code'],
                        'partsearch[do][partsearch_007]': '1',
                        'func': 'sendparttabsearch',
                        'giveback': json.dumps({}),
                        'api_json_request': '1'}

                b = 0
                while True:
                    b += 1
                    try:
                        resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                                             data=data,
                                             headers=get_header(),
                                             proxies=get_proxy(),
                                             timeout=(10, 10))

                        if resp.status_code == 200:
                            json_ = resp.json()
                            
                            if 'redirect_to_url' not in json_ and not resp.json()['giveback'] and 'collected_javascript' in json_:
                                break
                    except KeyboardInterrupt:
                        break
                    except:
                        continue
                            
                # = = = = = = = = = = = = = = =

                soup = BeautifulSoup(json_['searchnoderesults'], 'lxml')
                html = etree.HTML(str(soup))

                # = = = = = = = = = = = = = = =

                list_tbody = html.xpath('//tbody[contains(@class, "listing-inner") and (contains(@class, "altrow-a-1") or contains(@class, "altrow-a-0"))]')

                # = = = = = = = = = = = = = = =

                list_json = [json.loads(tbody.xpath('./tr[1]/td[contains(@class, "listing-inner-content")]/input[contains(@id, "listing_data_essential[__GIP__")]/@value')[0]) for tbody in list_tbody]

                # = = = = = = = = = = = = = = =

                import numpy as np

                array_index_type_code = np.where(np.array([json_['parttype'] for json_ in list_json]) == input_.loc[a, 'Type Code'])

                # = = = = = = = = = = = = = = =

                list_key = [json_['partkey'] for json_ in list_json]

                # = = = = = = = = = = = = = = =

                list_json = [json.loads(tbody.xpath('./tr[1]/td/input[contains(@id, "listing_data_supplemental[")]/@value')[0]) for tbody in list_tbody]

                # = = = = = = = = = = = = = = =

                array_index_part_number = np.where(np.array([json_['partnumber'] for json_ in list_json]) == input_.loc[a, 'Part Number'])

                # = = = = = = = = = = = = = = =

                array_index_manufacturer = np.where(np.array([json_['catalogname'] for json_ in list_json]) == input_.loc[a, 'Manufacturer'])

                # = = = = = = = = = = = = = = =

                list_parameter = [json_['paramdesc'].strip() for json_ in list_json]

                # = = = = = = = = = = = = = = =

                array_index_part = np.intersect1d(np.intersect1d(array_index_type_code, array_index_part_number), array_index_manufacturer)

                # = = = = = = = = = = = = = = =

                if len(array_index_part) == 1:
                    part_index = array_index_part[0]
                else:
                    raise

                # = = = = = = = = = = = = = = =

                list_note = list_tbody[part_index].xpath('./tr[1]/td/div[@class="listing-text-row-moreinfo-truck"]/span[@class="span-link-underline-remover"]/text()')
                note = '\n'.join([note.strip() for note in list_note])

                # = = = = = = = = = = = = = = =

                list_info = list_tbody[part_index].xpath('./tr[1]/td/div[@class="listing-text-row-moreinfo-truck"]/a[@class="ra-btn ra-btn-moreinfo"]/@href')
                info = list_info[0].strip() if list_info and list_info[0].startswith('https://www.rockauto.com/en/moreinfo.php?') else f'''https://www.rockauto.com/en/moreinfo.php?pk={list_key[part_index]}&pt={input_.loc[a, 'Type Code']}'''

                # = = = = = = = = = = = = = = =

                note_2 = etree.tostring(list_tbody[part_index].xpath('./tr[1]/td/div[@class="listing-text-row"]')[0]).decode('utf-8')
                note_2 = re.sub(r'<.*?>', '', note_2.strip())
                note_2 = re.sub(r' +', ' ', note_2.strip())

                # = = = = = = = = = = = = = = =

                oem = ','.join(list_tbody[part_index].xpath('./tr[1]/td/span[contains(@title, "Replaces these Alternate")]/text()'))
                list_oem = [oem.strip() for oem in oem.split(',')]
                oem = ';'.join(sorted(list_oem))

                # = = = = = = = = = = = = = = =

                list_src = json.loads(list_tbody[part_index].xpath('./tr[1]/td/div/input[contains(@id, "jsninlineimg[")]/@value')[0])['Slots'] if list_tbody[part_index].xpath('./tr[1]/td/div/input[contains(@id, "jsninlineimg[")]/@value') else []

                # = = = = = = = = = = = = = = =

                dict_src = {}
                for i, src in zip(range(len(list_src)), list_src):
                    dict_src[str(i)] = 'https://www.rockauto.com' + src['ImageData']['Full'].strip()

                # = = = = = = = = = = = = = = =

                price = etree.tostring(list_tbody[part_index].xpath('./tr[1]/td[contains(@id, "listingtd[") and contains(@id, "][price]")]/span/span/span[contains(@id, "dprice[") and contains(@id, "][v]")]')[0]).decode('utf-8')
                dict_price = {'0': re.sub(r'<.*?>', '', price.strip())}

                # = = = = = = = = = = = = = = =

                list_li = list_tbody[part_index].xpath('./tr[1]/td/div/div/span/span[contains(@id, "ddrepl[optionchoice[__")]/table/tr/td[1]/div/div/ul/li')
                if list_li:
                    dict_price['1'] = {}
                    for j, li in zip(range(len(list_li[1:])), list_li[1:]):
                        dict_price['1'][str(j)] = re.sub(r' +', ' ', re.sub(r'<.*?>', ' ', etree.tostring(li.xpath('./div[contains(@class, "outer")]/div[contains(@class, "inner")]')[0]).decode('utf-8'))).strip()

                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                                         'Group': input_.loc[a, 'Group'],
                                         'Type': input_.loc[a, 'Type'],
                                         'Type Code': input_.loc[a, 'Type Code'],
                                         'Manufacturer': input_.loc[a, 'Manufacturer'],
                                         'Part Number': input_.loc[a, 'Part Number'],
                                         'Parameter': list_parameter[part_index],
                                         'Json_Price': json.dumps(dict_price),
                                         'Note 1': note,
                                         'Note 2': note_2,
                                         'OEM': oem,
                                         'Picture': '',
                                         'Url': input_.loc[a, 'Url'],
                                         'Info': info,
                                         'Json_Src': json.dumps(dict_src),
                                         'Key': list_key[part_index]}])

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'
            except:
                output_error = pd.concat([output_error, pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and work.qsize() == 0 or work.qsize() % 1_000 == 0 and work.qsize() != 0:
                clear_output()

            print(f'''[状态：{crawler_status}，尝试次数：{b}] - {(list_file.index(file) + 1) / len(list_file) * 100:.2f}% - {file.removesuffix('.xlsx')} > {input_.loc[a, 'Type']} > {input_.loc[a, 'Manufacturer']} > {input_.loc[a, 'No']}.{input_.loc[a, 'Part Number']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

    # = = = = = = = = = = = = = = =

    list_task = []
    for _ in range(5):
        task = gevent.spawn(crawler)
        list_task.append(task)
    gevent.joinall(list_task)

    print('输出ing...')
    print()
    if not output_correct.empty:
        output_correct = output_correct.sort_values(by=['No'],
                                                    ascending=[True],
                                                    ignore_index=True)
        output_correct.to_excel(f'''./file/2.part/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if not output_error.empty:
        output_error = output_error.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
        output_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-part_error.xlsx''', index=False)
        print('爬虫存在error')
        print()

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/2.part'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

from tqdm import tqdm

for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'./file/2.part/{file}',
              f'''./file/2.part/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

总数量：507

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > Timing Chain & Component Kit > MELLING > 4.30430S
[剩余数量：477] - [当前时间：16:12:39]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > Timing Chain & Component Kit > MELLING > 19.31036S
[剩余数量：476] - [当前时间：16:12:39]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > Timing Chain & Component Kit > MELLING > 24.31045S
[剩余数量：475] - [当前时间：16:12:39]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > Timing Chain & Component Kit > MELLING > 23.31041S
[剩余数量：474] - [当前时间：16:12:39]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > Timing Chain & Component Kit > MELLING > 16.31007S
[剩余数量：473] - [当前时间：16:12:39]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > Timing Chain & Component Kit > MELLING > 18.31035SX
[剩余数量：472] - [当前时间：16:12:39]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > Timing Chain & Component Kit > MELLING > 17.31035S
[剩余数量：471] - [当前时间：16:12:39]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > Timing Chain & Component Kit > MELLING > 5.30460S
[剩余数量：470] - [当前时间：16:12:39]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > Ti

Progress: 100%|██████████████████████████████| 2/2 [00:00<00:00, 3443.60it/s]

Done ~
